In [1]:

# declaring the variables

# Sensor Configuration
## Profile config
start_freq = 77  # GHz 
slope = 79  # MHz/us
# idle_time = 5  # us
adc_start_time = 6  # us
adc_samples = 256  # Number of samples per chirp
sample_freq = 8000  # ksps (8 MSPS)
# ramp_end_time = 40  # us
rx_gain = 48  # dB

## Frame config
### Chirp config
start_chirp_tx = 0
end_chirp_tx = 11
nchirp_loops = 64
nframes_master = 0  # Number of Frames for Master (0 means continuous streaming mode)
nframes_slave = 0  # Number of Frames for Slaves (0 means continuous streaming mode)
Inter_Frame_Interval = 100  # ms
trigger_delay = 0  # us
nDummy_chirp = 0
trig_list = {1, 2, 2, 2}  # 1: Software trigger, 2: Hardware trigger 

# Derived config
numAdcSamples = adc_samples
numRxPerDevice = 4
numDevice = numRxPerDevice
numChirpsPerFrame = end_chirp_tx - start_chirp_tx + 1
numChirpLoops = nchirp_loops
numChirpsPerFramePerDevice = numChirpsPerFrame * numChirpLoops
totalSamplePerFramePerDevice = numChirpsPerFramePerDevice * numAdcSamples * numRxPerDevice  

print(f"{numAdcSamples}, {numRxPerDevice}, {numChirpsPerFramePerDevice}, {totalSamplePerFramePerDevice}") # checking the values


256, 4, 768, 786432


## Parsing the ADC data

parsing the binary adc data


In [5]:
import numpy as np
import os

def read_adc_data(file_path, num_adc_samples):
    adc_data = np.fromfile(file_path, dtype=np.int16)
    # Reshape the data into I and Q components
    adc_data = adc_data.reshape(-1, 2)
    adc_data_complex = adc_data[:, 0] + 1j * adc_data[:, 1]
    return adc_data_complex

def calculate_num_frames(file_path, num_chirps_per_frame, num_rx, num_adc_samples):
    file_size_bytes = os.path.getsize(file_path)
    # Each sample is 2 bytes for I and 2 bytes for Q = 4 bytes per complex sample
    bytes_per_sample = 4
    samples_per_frame = num_chirps_per_frame * num_rx * num_adc_samples
    bytes_per_frame = samples_per_frame * bytes_per_sample
    total_frames = file_size_bytes // bytes_per_frame
    return total_frames

def parse_device_data(device_name, data_directory, num_chirps_per_frame, num_rx, num_adc_samples):
    # Get a list of data files for the device
    data_files = [f for f in os.listdir(data_directory) if f.startswith(device_name) and '_data.bin' in f]
    data_files.sort()  # Ensure files are in order
    adc_data_list = []
    for file_name in data_files:
        file_path = os.path.join(data_directory, file_name)
        adc_data_complex = read_adc_data(file_path, num_adc_samples)
        num_frames = calculate_num_frames(file_path, num_chirps_per_frame, num_rx, num_adc_samples)
        # Reshape the data
        adc_data_complex = adc_data_complex.reshape(num_frames, num_chirps_per_frame, num_rx, num_adc_samples)
        adc_data_list.append(adc_data_complex)
    # Concatenate data from multiple files
    device_data = np.concatenate(adc_data_list, axis=0)  # Concatenate along frames
    return device_data


In [6]:
# Device data chunked

import numpy as np
import os

def parse_device_data_chunked(device_name, data_directory, num_chirps_per_frame, num_rx, num_adc_samples, chunk_size):
    # Get a list of data files for the device
    data_files = [f for f in os.listdir(data_directory) if f.startswith(device_name) and '_data.bin' in f]
    data_files.sort()  # Ensure files are in order
    adc_data_list = []
    for file_name in data_files:
        file_path = os.path.join(data_directory, file_name)
        file_size = os.path.getsize(file_path)
        total_int16_elements = file_size // 2
        total_complex_samples = total_int16_elements // 2  # I and Q

        # Create a memory-mapped array
        adc_data_int16 = np.memmap(file_path, dtype=np.int16, mode='r', shape=(total_int16_elements,))

        # Process data in chunks
        num_samples_per_frame = num_chirps_per_frame * num_rx * num_adc_samples
        total_frames = total_complex_samples // num_samples_per_frame

        num_chunks = (total_frames + chunk_size - 1) // chunk_size  # Ceiling division

        for chunk_idx in range(num_chunks):
            start_frame = chunk_idx * chunk_size
            end_frame = min((chunk_idx + 1) * chunk_size, total_frames)
            num_frames_in_chunk = end_frame - start_frame

            start_idx = start_frame * num_samples_per_frame * 2  # Multiply by 2 for I and Q
            end_idx = end_frame * num_samples_per_frame * 2

            # Access data for the chunk
            adc_data_chunk = adc_data_int16[start_idx:end_idx]

            # Reshape and convert to complex64
            adc_data_chunk = adc_data_chunk.reshape(-1, 2)
            adc_data_complex_chunk = np.empty(adc_data_chunk.shape[0], dtype=np.complex64)
            adc_data_complex_chunk.real = adc_data_chunk[:, 0].astype(np.float32)
            adc_data_complex_chunk.imag = adc_data_chunk[:, 1].astype(np.float32)

            # Reshape into [frames, chirps, rx, samples]
            adc_data_complex_chunk = adc_data_complex_chunk.reshape(
                num_frames_in_chunk, num_chirps_per_frame, num_rx, num_adc_samples
            )

            # Process the chunk (e.g., append to list or process immediately)
            adc_data_list.append(adc_data_complex_chunk)

    # Concatenate data from all chunks
    device_data = np.concatenate(adc_data_list, axis=0)  # Concatenate along frames
    return device_data


In [7]:
data_directory = "data/capture_drone_steady"

# Radar configuration parameters
num_chirps_per_frame = 768
num_rx = 4
num_adc_samples = 256

# Chirp timing parameters
idle_time = 5e-6  # 5 μs
ramp_end_time = 40e-6  # 40 μs
Tc = idle_time + ramp_end_time  # Chirp duration in seconds
fd = 1 / Tc  # Doppler sampling frequency

devices = ['master', 'slave1', 'slave2', 'slave3']
device_data = {}

for device in devices:
    print(f"Parsing data for {device}...")
    device_data[device] = parse_device_data_chunked(
        device_name=device,
        data_directory=data_directory,
        num_chirps_per_frame=num_chirps_per_frame,
        num_rx=num_rx,
        num_adc_samples=num_adc_samples,
        chunk_size=10  # Number of frames per chunk
    )



Parsing data for master...


MemoryError: Unable to allocate 7.22 GiB for an array with shape (1232, 768, 4, 256) and data type complex64

In [1]:
# Chirp configuration mapping

chirp_tx_mapping = {
    0: {'device': 'slave3', 'tx': 'TX2'},
    1: {'device': 'slave3', 'tx': 'TX1'},
    2: {'device': 'slave3', 'tx': 'TX0'},
    3: {'device': 'slave2', 'tx': 'TX2'},
    4: {'device': 'slave2', 'tx': 'TX1'},
    5: {'device': 'slave2', 'tx': 'TX0'},
    6: {'device': 'slave1', 'tx': 'TX2'},
    7: {'device': 'slave1', 'tx': 'TX1'},
    8: {'device': 'slave1', 'tx': 'TX0'},
    9: {'device': 'master', 'tx': 'TX2'},
    10: {'device': 'master', 'tx': 'TX1'},
    11: {'device': 'master', 'tx': 'TX0'},
}

# virtual antenna array

def construct_virtual_array(device_data, num_frames, num_chirps_per_frame, num_rx, num_adc_samples):
    num_devices = 4
    num_tx_antennas = 3  # Each device has 3 TX antennas
    num_virtual_antennas = num_devices * num_tx_antennas * num_rx  # 4 devices * 3 TX * 4 RX = 48

    virtual_data = np.zeros((num_frames, num_chirps_per_frame, num_rx, num_adc_samples), dtype=complex)

    for chirp_idx in range(num_chirps_per_frame):
        mapping = chirp_tx_mapping[chirp_idx % 12]  # There are 12 unique chirps
        device = mapping['device']
        # Get data for the current chirp from the corresponding device
        virtual_data[:, chirp_idx, :, :] = device_data[device][:, chirp_idx, :, :]

    return virtual_data

num_frames = device_data['master'].shape[0]  # Assuming all devices have the same number of frames

virtual_data = construct_virtual_array(device_data, num_frames, num_chirps_per_frame, num_rx, num_adc_samples)


NameError: name 'device_data' is not defined

In [ ]:
# Signal Processing

## Range FFT
def range_fft(virtual_data, num_adc_samples):
    # Apply window function (e.g., Hanning window)
    window = np.hanning(num_adc_samples)
    virtual_data_windowed = virtual_data * window[np.newaxis, np.newaxis, np.newaxis, :]

    # Perform FFT along the samples dimension
    range_profiles = np.fft.fft(virtual_data_windowed, n=num_adc_samples, axis=-1)
    return range_profiles

# range profile
range_profiles = range_fft(virtual_data, num_adc_samples)

## Doppler FFT
def doppler_fft(range_profiles, num_chirps_per_frame):
    num_frames, num_chirps, num_rx, num_range_bins = range_profiles.shape
    # Reshape to combine frames and chirps for Doppler processing
    reshaped_data = range_profiles.reshape(num_frames * num_chirps, num_rx, num_range_bins)

    # Apply window function along the chirp dimension
    window = np.hanning(num_chirps_per_frame)
    window = np.tile(window, num_frames)
    window = window[:, np.newaxis, np.newaxis]

    reshaped_data_windowed = reshaped_data * window

    # Perform FFT along the chirp dimension
    doppler_fft_size = num_chirps_per_frame
    doppler_spectrum = np.fft.fftshift(np.fft.fft(reshaped_data_windowed, n=doppler_fft_size, axis=0), axes=0)

    # Reshape back to original dimensions
    doppler_spectrum = doppler_spectrum.reshape(num_frames, num_chirps_per_frame, num_rx, num_range_bins)
    return doppler_spectrum

# doppler profile
doppler_spectrum = doppler_fft(range_profiles, num_chirps_per_frame)



In [ ]:
# time for plotting doppler spectrum

range_magnitude = np.abs(range_profiles)
range_magnitude_mean = np.mean(range_magnitude, axis=(0, 2))  # Average over frames and RX antennas

# Sum over chirps
range_profile = np.sum(range_magnitude_mean, axis=0)

import matplotlib.pyplot as plt

plt.figure()
plt.plot(range_profile)
plt.title('Range Profile')
plt.xlabel('Range Bin')
plt.ylabel('Amplitude')
plt.show()

# Manually select the range bin corresponding to the drone
selected_range_bin = int(input("Enter the index of the range bin corresponding to the drone: "))

# ------------------------------------------------------------------------------------------------------------------------------------------------------------
# Generate Frequency Axis

# Extract the Doppler spectrum for the selected range bin and average over RX antennas
doppler_spectrum_selected = doppler_spectrum[:, :, :, selected_range_bin]
doppler_spectrum_mean = np.mean(doppler_spectrum_selected, axis=2)  # Average over RX antennas

# Sum over frames
doppler_spectrum_sum = np.sum(doppler_spectrum_mean, axis=0)


In [ ]:
import matplotlib.pyplot as plt

# Now process the data in chunks
doppler_spectrum_accum = None  # Initialize accumulator

for chunk_idx in range(num_chunks):
    # Extract the chunk data for each device
    chunk_device_data = {device: device_data[device][chunk_idx] for device in devices}
    
    # Construct virtual antenna array for the chunk
    virtual_data_chunk = construct_virtual_array(chunk_device_data, ...)
    
    # Perform Range FFT
    range_profiles_chunk = range_fft(virtual_data_chunk, num_adc_samples)
    
    # Identify range bin of the drone (if necessary, use the first chunk to select the range bin)
    if chunk_idx == 0:
        # Compute range profile and select range bin as before
        selected_range_bin = ...  # As before
    
    # Perform Doppler FFT
    doppler_spectrum_chunk = doppler_fft(range_profiles_chunk, num_chirps_per_frame)
    
    # Extract Doppler spectrum for the selected range bin
    doppler_spectrum_selected = doppler_spectrum_chunk[:, :, :, selected_range_bin]
    doppler_spectrum_mean = np.mean(doppler_spectrum_selected, axis=2)  # Average over RX antennas
    doppler_spectrum_sum = np.sum(doppler_spectrum_mean, axis=0)  # Sum over frames in the chunk
    
    # Accumulate Doppler spectrum
    if doppler_spectrum_accum is None:
        doppler_spectrum_accum = doppler_spectrum_sum
    else:
        doppler_spectrum_accum += doppler_spectrum_sum

# After processing all chunks, normalize and plot the Doppler spectrum
amplitude_spectrum = np.abs(doppler_spectrum_accum)

freq_bins = np.fft.fftshift(np.fft.fftfreq(num_chirps_per_frame, d=Tc))

plt.figure()
plt.plot(freq_bins, amplitude_spectrum)
plt.title('Doppler Spectrum (Frequency vs. Amplitude)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()